In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
import pickle
import pandas as pd
import time
import csv

In [7]:
# pre
data_dir = "data/"
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    

test_df = load_obj("test_df")

# Get the training_data_df
training_data_df = load_obj("training_df_20w_ra")
final_training_data_df = training_data_df.iloc[:,3:20]
# 0.511756 0.028259
# get the labeled data df
final_labels_df = training_data_df.iloc[:,2]
print(final_labels_df)
test_df = test_df.iloc[:,2:20]

0    0.0
0    0.0
0    0.0
0    0.0
0    0.0
    ... 
0    1.0
0    1.0
0    1.0
0    1.0
0    1.0
Name: 2, Length: 200000, dtype: float64


In [12]:
print(final_training_data_df)

print(test_df)


           3         4         5          6          7         8
0   0.000000  0.002924  0.005814      342.0   0.090700  0.000016
0   0.000000  0.000000  0.000000     9504.0   0.000000  0.000000
0   0.000000  0.250000  0.250000        4.0   0.073830  0.000001
0   0.000000  0.000000  0.000000      116.0   0.000000  0.000000
0   0.000000  0.000000  0.000000        5.0   0.000000  0.000000
..       ...       ...       ...        ...        ...       ...
0   0.562058  0.000008  0.000480  5812404.0   4.631456  0.002710
0   0.661151  0.000035  0.020520  4760100.0  18.150964  0.031308
0   0.000000  0.000000  0.000000    45891.0   0.000000  0.000000
0   0.000000  0.000000  0.000000    66889.0   0.000000  0.000000
0   0.545567  0.000007  0.000300  4514305.0   3.045588  0.000872

[200000 rows x 6 columns]
           2         3         4        5         6         7
0   0.000000  0.000000  0.000000    667.0  0.000000  0.000000
0   0.000000  0.000000  0.000000    666.0  0.000000  0.000000
0   0.6

In [15]:
# final_training_data_df = rescale_min_max(final_training_data_df)
X=final_training_data_df
# count=0
# get the data and label
y=final_labels_df

from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X,y)
x_train = X_t
x_test = X_test
y_train = y_t
y_test = y_test

In [16]:
print(x_train)

           3         4         5          6         7         8
0   0.000000  0.002830  0.068966     2120.0  0.521838  0.000062
0   0.000000  0.000000  0.000000       15.0  0.000000  0.000000
0   0.000000  0.000000  0.000000     9390.0  0.000000  0.000000
0   0.559540  0.000010  0.000551  3789005.0  3.580236  0.002010
0   0.000000  0.000000  0.000000      368.0  0.000000  0.000000
..       ...       ...       ...        ...       ...       ...
0   0.000000  0.000137  0.010400   145470.0  1.787724  0.000343
0   0.000000  0.000000  0.000000   743342.0  0.000000  0.000000
0   0.651377  0.000033  0.014726  1297086.0  6.068562  0.083869
0   0.000000  0.000000  0.000000      256.0  0.000000  0.000000
0   0.000000  0.000000  0.000000       12.0  0.000000  0.000000

[150000 rows x 6 columns]


In [25]:
# x_train = np.random.random((1000, 20))
# y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
# x_test = np.random.random((100, 20))
# y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
model.add(Dense(16, input_dim=6, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=8,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/8
1172/1172 [==============================] - 1s 624us/step - loss: 28237.0586 - accuracy: 0.6223
Epoch 2/8
1172/1172 [==============================] - 1s 557us/step - loss: 34.8976 - accuracy: 0.6284
Epoch 3/8
1172/1172 [==============================] - 1s 561us/step - loss: 2.2730 - accuracy: 0.7862
Epoch 4/8
1172/1172 [==============================] - 1s 560us/step - loss: 1.0737 - accuracy: 0.8172
Epoch 5/8
1172/1172 [==============================] - 1s 557us/step - loss: 0.7207 - accuracy: 0.8226
Epoch 6/8
1172/1172 [==============================] - 1s 576us/step - loss: 0.7205 - accuracy: 0.8304
Epoch 7/8
1172/1172 [==============================] - 1s 561us/step - loss: 1.2211 - accuracy: 0.8351
Epoch 8/8
391/391 [==============================] - 0s 371us/step - loss: 0.3165 - accuracy: 0.9060


In [26]:
test_df.shape
test_data_df = test_df

In [27]:

from tqdm import tqdm
def predict():
    """
    make the prediction using the jaccard's coefficient
    """
    result = []
    cnt=0
    for row in tqdm(test_data_df.iterrows()):
        row_df = pd.DataFrame(row[1]).T
        single_result = model.predict(row_df)[0][0]
        #print(model.predict(row_df)[0][0])
        cnt+=1
        result.append((cnt, single_result))
    return result
result = predict()

# # save the result


'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())


"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
    headers = ['id','Predicted']

    with open(filename + str(nowtime()) + ".csv", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(result)
save_prediction_to_csv(result, "shawn_ann_v1_200epoch")

2000it [01:03, 31.27it/s]
